In [1]:
import pandas as pd 
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_1_1.csv')
#print(df.head(2))

print('Q1-1:', df.groupby('대륙')['맥주소비량'].mean().idxmax())
df_target = df[df['대륙'] == df.groupby('대륙')['맥주소비량'].mean().idxmax()]
print('Q1-2:', df_target.groupby('국가')['맥주소비량'].sum().sort_values(ascending=False).reset_index().iloc[4]['국가'])
print('Q1-3:', 
      round(df[df['국가'] == 
      df_target.groupby('국가')['맥주소비량'].sum().sort_values(ascending=False).reset_index().iloc[4]['국가']]['맥주소비량'].mean(), 0))


df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_1_2.csv')
df['합계'] = df['관광'] + df['사무'] + df['공무'] + df['유학'] + df['기타']
df['비율'] = df['관광'] / df['합계']
print('Q2-1:', df[df['국가'] == df.groupby('국가')['비율'].mean().sort_values(ascending=False).reset_index().iloc[1]['국가']]['관광'].item())
print('Q2-2:', df[df['국가'] == df.groupby('국가')['관광'].sum().sort_values(ascending=False).reset_index().iloc[1]['국가']]['공무'].mean())
print('Q2-3:', df[df['국가'] == df.groupby('국가')['비율'].mean().sort_values(ascending=False).reset_index().iloc[1]['국가']]['관광'].item() +
      df[df['국가'] == df.groupby('국가')['관광'].sum().sort_values(ascending=False).reset_index().iloc[1]['국가']]['공무'].mean())

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_1_3.csv')
print(df.head(2))
df['CO(GT)'] = (df['CO(GT)'] -  df['CO(GT)'].min()) / (df['CO(GT)'].max() - df['CO(GT)'].min())
df['NMHC(GT)'] = (df['NMHC(GT)'] -  df['NMHC(GT)'].min()) / (df['NMHC(GT)'].max() - df['NMHC(GT)'].min())
print('Q3')
print('CO(GT) std:', round(df['CO(GT)'].std(),2))
print('NMHC(GT) std:', round(df['NMHC(GT)'].std(),2))

Q1-1: SA
Q1-2: Venezuela
Q1-3: 253.0
Q2-1: 9039
Q2-2: 493.8
Q2-3: 9532.8
         Date      Time  CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)
0  13/10/2004  06.00.00  -200.0        798.0    -200.0       2.2
1  31/03/2005  10.00.00     1.3        996.0    -200.0       4.9
Q3
CO(GT) std: 0.37
NMHC(GT) std: 0.15


In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_2_test.csv')

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

key = test['ID']
y_train = train['count']
x_train = train.drop(['ID', 'count'], axis=1)
x_test = test.drop(['ID', 'count'], axis=1)

concat = pd.concat([x_train, x_test], axis=0)

obj = ['holiday', 'workingday', 'weather']
num = ['temp', 'atemp', 'humidity', 'windspeed']

concat = pd.get_dummies(concat, columns = ['weather'])
bo = ['holiday', 'workingday']

for col in bo:
    concat[col] = concat[col].map({'Yes':True, 'No':False})
    
#scaler = StandardScaler()
scaler = MinMaxScaler()
concat[num] = scaler.fit_transform(concat[num])

x_train = concat.iloc[:len(x_train), :]
x_test = concat.iloc[len(x_train):, :]

x_t, x_va, y_t, y_va = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

model = RandomForestRegressor(
    n_estimators = 1000,
    max_depth = 5,
    min_samples_leaf = 3,
    min_samples_split = 4,
    random_state = 42,
)

model.fit(x_t, y_t)
pred = model.predict(x_va)
print('MAE:', mean_absolute_error(y_va, pred))

result = pd.DataFrame()
result['ID'] = key
result['pred'] = model.predict(x_test)
print(result.head(5))

MAE: 118.65697648613416
      ID        pred
0   4775  107.251103
1  10539  223.901097
2   8229   90.055812
3   8677  279.875737
4   5071  136.188307


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_3_1.csv')
print('before:', round(df.before.mean(),2), round(df.before.std(),2))
print('after:', round(df.after.mean(),2), round(df.after.std(),2))

from statsmodels.stats.weightstats import ttest_ind
from scipy.stats import wilcoxon
wil_stats, p_value = wilcoxon(df['before'], df['after'])
print('Pr:', round(p_value, 2))
print('Stats:', wil_stats)

train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_3_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/8_3_2_test.csv')

import statsmodels.formula.api as smf

model = smf.ols('productivity ~ hours + age + experience', data=train).fit()
#print(model.params.sort_values(ascending=False).idxmax())
print(model.params[model.params.index != 'Intercept'].idxmax())
print(len(model.params[model.pvalues < 0.05])-1)
pred = model.predict(test)
from scipy.stats import pearsonr
print((pearsonr(pred, test['productivity'])[0])**2)

before: 8.21 1.71
after: 7.23 1.96
Pr: 0.0
Stats: 72.0
hours
3
0.8081550160783021
